<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=da3fa1df4d1462c5321dab9e793c5152d1a939c9fc653c5b8659f8c3c5a8104b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-13 13:00:31
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.50 L
Current:  1.45 C
-------------------
Today PnL: 10.35 K (0.07%)
Current PnL: -21.34 L (-13.99%)
CY Booked + Current PnL: -7.82 L (-5.13%)
-------------------
Total profit:  2.07 L
Total loss:  -23.42 L
-------------------
Total Booked + Current PnL: 19.16 L (15.25%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.31%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 88.01 L (60.58%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.46%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,101.92,52.0,M-SC,2.70,87010.0,-13767.0,13861.0,0.03,-13.66,15.93,0.09,245.0,-0.99,0.61,14.31,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.65,45.0,X-MC,3.37,88200.0,-2478.0,17384.0,-0.57,-2.73,19.71,16.44,101.0,-0.14,0.61,16.89,X40,ATH,PHARMA
38,INDIGOPNTS,1408.00,139.40,82.0,M-SC,3.23,157139.0,-17420.0,17458.0,2.82,-9.98,11.11,0.02,221.0,-1.00,1.09,36.68,OX40N,NTT,PAINTS
34,ICICIGI,2252.93,-12.45,59.0,X-MC,7.58,189255.0,14935.0,20269.0,0.32,8.57,10.71,20.19,91.0,0.74,1.32,24.98,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-2.24,66.0,X-LC,8.75,300667.0,35241.0,21558.0,0.52,13.28,7.17,21.40,11.0,1.63,2.09,20.80,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.27,85.0,X-LC,7.97,253009.0,1241.0,51968.0,3.80,0.49,20.54,21.13,27.0,0.02,1.76,36.09,X40,ATH,PAINTS
13,BERGEPAINT,680.00,-13.11,70.0,X-MC,1.62,233685.0,6346.0,41713.0,3.02,2.79,17.85,21.14,106.0,0.15,1.63,31.72,XY24,NTT,PAINTS
38,INDIGOPNTS,1408.00,139.40,82.0,M-SC,3.23,157139.0,-17420.0,17458.0,2.82,-9.98,11.11,0.02,221.0,-1.00,1.09,36.68,OX40N,NTT,PAINTS
5,ANGELONE,3033.00,27.04,74.0,X-SC,7.11,225106.0,34100.0,23591.0,2.45,17.85,10.48,30.21,157.0,1.45,1.57,42.46,X40N,NTT,FINANCE
46,KANSAINER,340.00,-67.24,56.0,H-SC,2.45,226251.0,-43416.0,79753.0,2.39,-16.10,35.25,13.47,138.0,-0.54,1.57,15.04,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-54.55,54.0,H-SC,9.51,259653.0,-26852.0,284190.0,-4.41,-9.37,109.45,89.82,134.0,-0.09,1.81,37.38,XY24,NTT,FINANCE
7,ASIANTILES,137.00,7538.89,64.0,L-SC,15.82,83821.0,-9989.0,86604.0,-1.85,-10.65,103.32,81.67,269.0,-0.12,0.58,62.32,XR,NTT,CERAMICS
15,CAMPUS,393.00,-26.15,40.0,M-SC,4.09,153914.0,-13861.0,69708.0,-1.85,-8.26,45.29,33.28,210.0,-0.20,1.07,19.29,XY24,NTT,FOOTWEAR
47,KPIGREEN,731.05,0.96,40.0,H-SC,10.23,120204.0,-5093.0,64021.0,-1.83,-4.06,53.26,47.03,141.0,-0.08,0.84,50.51,MH,ATH,POWER
54,QUESS,986.00,-52.62,30.0,X-SC,37.27,47254.0,-17752.0,168683.0,-1.65,-27.31,356.97,232.18,198.0,-0.11,0.33,0.00,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152322.0,1377.0,108072.0,0.19,0.91,70.95,72.51,53.0,0.01,1.06,6.99,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.48,38.0,H-MC,5.89,208900.0,-674.0,75350.0,-0.65,-0.32,36.07,35.63,NaN,-0.01,1.45,16.30,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIGOPNTS,1408.0,139.40,82.0,M-SC,3.23,157139.0,-17420.0,17458.0,2.82,-9.98,11.11,0.02,221.0,-1.00,1.09,36.68,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.0,101.92,52.0,M-SC,2.70,87010.0,-13767.0,13861.0,0.03,-13.66,15.93,0.09,245.0,-0.99,0.61,14.31,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.24,56.0,H-SC,2.45,226251.0,-43416.0,79753.0,2.39,-16.10,35.25,13.47,138.0,-0.54,1.57,15.04,XY24,NTT,PAINTS
66,SIS,528.0,2027.46,48.0,H-SC,2.41,85858.0,-25174.0,48785.0,-0.06,-22.67,56.82,21.26,156.0,-0.52,0.60,16.00,OX40N,NTT,MISC
72,TATAELXSI,9161.0,-24.89,47.0,H-MC,6.30,101707.0,-30128.0,72354.0,-0.65,-22.85,71.14,32.03,98.0,-0.42,0.71,13.92,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152322.0,1377.0,108072.0,0.19,0.91,70.95,72.51,53.0,0.01,1.06,6.99,XR,NTT,IT
37,INDIAMART,4810.62,-52.96,55.0,H-SC,4.76,131080.0,7744.0,123884.0,-1.25,6.28,94.51,106.72,119.0,0.06,0.91,31.12,AR,ATH,MISC
61,SAMMAANCAP,326.00,-193.18,56.0,M-SC,36.11,160713.0,11493.0,132685.0,0.90,7.70,82.56,96.62,208.0,0.09,1.12,73.93,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4810.62,-52.96,55.0,H-SC,4.76,131080.0,7744.0,123884.0,-1.25,6.28,94.51,106.72,119.0,0.06,0.91,31.12,AR,ATH,MISC
84,WIPRO,420.00,-13.34,55.0,M-LC,5.45,152322.0,1377.0,108072.0,0.19,0.91,70.95,72.51,53.0,0.01,1.06,6.99,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-9.48,38.0,H-MC,5.89,208900.0,-674.0,75350.0,-0.65,-0.32,36.07,35.63,NaN,-0.01,1.45,16.30,AR,ATH,PHARMA
83,WHIRLPOOL,2270.00,-45.97,35.0,M-SC,9.14,111964.0,-2834.0,96871.0,-0.81,-2.47,86.52,81.92,223.0,-0.03,0.78,32.55,XR,NTT,DURABLES
47,KPIGREEN,731.05,0.96,40.0,H-SC,10.23,120204.0,-5093.0,64021.0,-1.83,-4.06,53.26,47.03,141.0,-0.08,0.84,50.51,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-17.97,27.0,X-MC,5.79,252105.0,-1791.0,84102.0,-1.25,-0.71,33.36,32.42,105.0,-0.02,1.75,14.29,X40,ATH,FMCG
1,ABB,7934.00,-43.56,29.0,H-LC,5.17,235968.0,-25651.0,144861.0,-0.16,-9.80,61.39,45.57,7.0,-0.18,1.64,0.41,AR,NTT,ELECTRICAL
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87108.0,-41562.0,86855.0,-0.76,-32.30,99.71,35.20,219.0,-0.48,0.61,0.25,X40,NTT,FOOTWEAR
54,QUESS,986.00,-52.62,30.0,X-SC,37.27,47254.0,-17752.0,168683.0,-1.65,-27.31,356.97,232.18,198.0,-0.11,0.33,0.00,XY24,NTT,MISC
45,JSWINFRA,342.00,-24.37,32.0,X-MC,5.04,189743.0,-10604.0,41098.0,-1.25,-5.29,21.66,15.23,178.0,-0.26,1.32,25.25,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-38.34,46.0,X-LC,0.88,198999.0,-1139.0,22029.0,-0.01,-0.57,11.07,10.44,4.0,-0.05,1.38,4.96,X40,NTT,FMCG
53,PGHH,17907.65,-32.93,32.0,X-MC,1.55,194100.0,-6720.0,74515.0,0.54,-3.35,38.39,33.76,80.0,-0.09,1.35,1.68,X40,ATH,FMCG
65,SIEMENS,4671.50,-4.11,47.0,H-LC,1.64,154225.0,-31870.0,79349.0,0.91,-17.13,51.45,25.51,15.0,-0.40,1.07,13.65,AR,ATH,ELECTRICAL
30,HAVELLS,2069.16,-7.74,44.0,X-MC,3.21,307860.0,-20131.0,126654.0,0.55,-6.14,41.14,32.48,92.0,-0.16,2.14,4.59,X40,ATH,ELECTRICAL
78,UNITDSPR,1644.00,-6.09,62.0,X-LC,3.28,246063.0,11919.0,36713.0,-0.35,5.09,14.92,20.77,86.0,0.32,1.71,12.36,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.62,30.0,X-SC,37.27,47254.0,-17752.0,168683.0,-1.65,-27.31,356.97,232.18,198.0,-0.11,0.33,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87108.0,-41562.0,86855.0,-0.76,-32.30,99.71,35.20,219.0,-0.48,0.61,0.25,X40,NTT,FOOTWEAR
1,ABB,7934.00,-43.56,29.0,H-LC,5.17,235968.0,-25651.0,144861.0,-0.16,-9.80,61.39,45.57,7.0,-0.18,1.64,0.41,AR,NTT,ELECTRICAL
53,PGHH,17907.65,-32.93,32.0,X-MC,1.55,194100.0,-6720.0,74515.0,0.54,-3.35,38.39,33.76,80.0,-0.09,1.35,1.68,X40,ATH,FMCG
10,BAJAJHFL,181.50,-16.17,37.0,X-MC,6.42,176792.0,-23714.0,120873.0,-0.21,-11.83,68.37,48.45,90.0,-0.20,1.23,1.95,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-52.62,30.0,X-SC,37.27,47254.0,-17752.0,168683.0,-1.65,-27.31,356.97,232.18,198.0,-0.11,0.33,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.65,42.0,X-SC,3.58,80840.0,-64320.0,140249.0,-1.06,-44.31,173.49,52.31,136.0,-0.46,0.56,8.21,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.65,45.0,X-MC,3.37,88200.0,-2478.0,17384.0,-0.57,-2.73,19.71,16.44,101.0,-0.14,0.61,16.89,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-41.72,30.0,X-SC,8.93,87108.0,-41562.0,86855.0,-0.76,-32.30,99.71,35.20,219.0,-0.48,0.61,0.25,X40,NTT,FOOTWEAR
33,HONAUT,58357.33,-24.48,36.0,X-SC,0.97,106500.0,-21438.0,68575.0,0.01,-16.76,64.39,36.84,143.0,-0.31,0.74,9.03,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-26.44,61.0,X-LC,8.78,293073.0,-52883.0,119574.0,-0.45,-15.29,40.80,19.28,1.0,-0.44,2.04,8.34,X40,ATH,IT
40,INFY,2275.00,-15.29,67.0,X-LC,4.32,330384.0,17496.0,154190.0,-0.04,5.59,46.67,54.87,3.0,0.11,2.30,14.34,X40,BTT,IT
42,ITC,452.00,-38.34,46.0,X-LC,0.88,198999.0,-1139.0,22029.0,-0.01,-0.57,11.07,10.44,4.0,-0.05,1.38,4.96,X40,NTT,FMCG
81,VBL,671.64,-18.84,46.0,X-LC,5.85,292555.0,-23287.0,135950.0,-0.16,-7.37,46.47,35.67,5.0,-0.17,2.03,5.46,X40N,ATH,FMCG
1,ABB,7934.00,-43.56,29.0,H-LC,5.17,235968.0,-25651.0,144861.0,-0.16,-9.80,61.39,45.57,7.0,-0.18,1.64,0.41,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.00,305.46,79.0,L-MC,6.69,49747.0,-27460.0,52851.0,0.97,-35.57,106.24,32.89,258.0,-0.52,0.35,37.00,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,48.0,L-SC,22.28,78720.0,-34829.0,74878.0,1.36,-30.67,95.12,35.27,268.0,-0.47,0.55,92.67,XR,NTT,HOTELS
7,ASIANTILES,137.00,7538.89,64.0,L-SC,15.82,83821.0,-9989.0,86604.0,-1.85,-10.65,103.32,81.67,269.0,-0.12,0.58,62.32,XR,NTT,CERAMICS
50,MASFIN,398.61,-19.15,49.0,H-SC,7.51,91800.0,-6180.0,27779.0,-0.65,-6.31,30.26,22.05,152.0,-0.22,0.64,33.44,XR,ATH,FINANCE
37,INDIAMART,4810.62,-52.96,55.0,H-SC,4.76,131080.0,7744.0,123884.0,-1.25,6.28,94.51,106.72,119.0,0.06,0.91,31.12,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANPAINT,3465.66,0.27,85.0,X-LC,7.97,253009.0,1241.0,51968.0,3.80,0.49,20.54,21.13,27.0,0.02,1.76,36.09,X40,ATH,PAINTS
79,VAIBHAVGBL,521.00,74.10,62.0,H-SC,7.59,150785.0,-31990.0,143065.0,0.92,-17.50,94.88,60.77,125.0,-0.22,1.05,35.79,XR,NTT,JEWELLERY
38,INDIGOPNTS,1408.00,139.40,82.0,M-SC,3.23,157139.0,-17420.0,17458.0,2.82,-9.98,11.11,0.02,221.0,-1.00,1.09,36.68,OX40N,NTT,PAINTS
7,ASIANTILES,137.00,7538.89,64.0,L-SC,15.82,83821.0,-9989.0,86604.0,-1.85,-10.65,103.32,81.67,269.0,-0.12,0.58,62.32,XR,NTT,CERAMICS
5,ANGELONE,3033.00,27.04,74.0,X-SC,7.11,225106.0,34100.0,23591.0,2.45,17.85,10.48,30.21,157.0,1.45,1.57,42.46,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.85
1,25,44.67
2,50,76.33


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.44
LC    28.01
MC    27.10
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.45
X40      20.83
X40N     12.40
XY25     10.17
XR        9.75
AR        9.08
OX40N     6.59
X200      1.85
SR        1.04
MH        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.82
X-MC    22.98
X-LC    20.02
M-SC    12.53
X-SC     5.63
H-LC     4.56
H-MC     3.77
M-LC     2.44
L-SC     1.46
L-LC     0.99
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.82,-4.04,37.68
IT,12.78,-14.46,75.73
FINANCE,12.06,-9.09,58.53
MISC,7.67,-17.81,77.24
PAINTS,6.05,-6.12,21.94
ELECTRICAL,5.59,-12.32,52.13
INSURANCE,4.50,2.24,32.22
PHARMA,3.46,-0.14,33.74
AUTO,3.17,-52.60,115.78


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3217679.0,20
AR,1264822.0,10
XR,1256059.0,13
X40,973404.0,14
X40N,685605.0,9
OX40N,538157.0,9
XY25,472369.0,7
SR,270571.0,2
MH,64021.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3404811.0,24
X-MC,1384370.0,16
M-SC,1244341.0,15
X-LC,772576.0,11
X-SC,540943.0,6
H-MC,329783.0,3
H-LC,282957.0,3
L-SC,250774.0,3
M-LC,154297.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1178175.0      6
           AR         841098.0      5
           XR         789065.0      7
M-SC       XY24       728273.0      6
X-MC       XY24       550066.0      4
           X40        487285.0      8
X-LC       X40        399264.0      5
X-SC       X40N       285405.0      4
H-SC       SR         270571.0      2
           OX40N      261881.0      3
X-LC       X40N       238229.0      3
H-LC       AR         224210.0      2
X-MC       XY25       185048.0      2
H-MC       XY24       182079.0      1
X-SC       XY24       168683.0      1
X-MC       X40N       161971.0      2
L-SC       XR         161482.0      2
M-SC       XR         144589.0      2
           XY25       132685.0      1
           AR         124164.0      2
           OX40N      114630.0      4
M-LC       XR         108072.0      1
L-SC       OX40N       89292.0      1
X-SC       X40         86855.0      1
H-MC       AR          75350.0      1
           OX40N       72354.0      1
X-LC       XY25        71711.0      2
H-SC       MH          64021.0      1
X-LC       XY24        63372.0      1
H-LC       X200        58747.0      1
L-MC       XR          52851.0      1
M-LC       XY25        46225.0      1
L-LC       XY25        36700.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
